This notebook is copied from 
https://www.kaggle.com/cdeotte/customers-who-bought-this-frequently-buy-this 

Then change the code to generate item-pair dictionary in purpose to calculate MAP@12 of validation data in https://www.kaggle.com/hervind/h-m-calculate-map-12-from-public-notebooks using algorithm in https://www.kaggle.com/cdeotte/recommend-items-purchased-together-0-021

Since the data is to large and takes ~7 hours to compute in Kaggle Kernels, I reduce the data period only last 3 months before validation date. 

So it will only get item-pairs of the transaction between '2020-06-16' and '2020-09-15'

# Customers Who Bought This Frequently Buy This!
In this notebook we will explore which items were frequently purchased together. Using this information, we can predict which items a customer will buy after we observe what they have already bought!

In [ ]:
import cudf, gc
import cv2, matplotlib.pyplot as plt
from os.path import exists
print('RAPIDS version',cudf.__version__)

from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool
from functools import partial
import numpy as np 

# Load Transactions

In [ ]:
# LOAD TRANSACTIONS DATAFRAME
df = cudf.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
print('Transactions shape',df.shape)

# REDUCE MEMORY OF DATAFRAME
df = df.loc[((df['t_dat'] >= '2020-06-16') & (df['t_dat'] < '2020-09-16'))] 
df = df[['customer_id','article_id']]
df.customer_id = df.customer_id.str[-16:].str.hex_to_int().astype('int64')
df.article_id = df.article_id.astype('int32')
print('Reduced Transactions shape',df.shape)
display( df.head() )
_ = gc.collect()

Created by my notebook here. Just change the line pairs[i.item()] = [vc2.index[0], vc2.index[1], vc2.index[2]] into pairs[i.item()] = vc2.index[0] and run the for-loop over the entire list of items with for j,i in enumerate(vc.index.values):

# Find Items Purchased Together
We will use RAPID cuDF to speed up the dataframe search commands below

In [ ]:
# FIND ITEMS PURCHASED TOGETHER
vc = df.article_id.value_counts()
list_articles = vc.index.values

pairs = {}
for i in tqdm(list_articles):
    df_u_temp = df.loc[df.article_id==i.item(),'customer_id']
    if len(df_u_temp) > 1: 
        USERS = df_u_temp.unique()
        df_filter = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id']
        if len(df_filter) == 1: 
            pairs[i.item()] = df_filter.values[0][0].item()
        elif len(df_filter) == 0: 
            continue
        else:     
            vc2 = df_filter.value_counts()    
            pairs[i.item()] = vc2.index[0]

In [ ]:
np.save("item_pair_3_months.npy", pairs)